In [2]:
# Imports
# Basics
from __future__ import print_function, division
import pandas as pd 
import numpy as np
import random
from matplotlib import pyplot as plt
%matplotlib inline
# gensim
# from gensim import corpora, models, similarities, matutils
# from gensim.models import Word2Vec

# nltk
from nltk.corpus import gutenberg

import sys

# sklearn

# keras
np.random.seed(13)
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Activation, SimpleRNN, GRU, LSTM, Convolution1D, MaxPooling1D, Merge, Dropout
from IPython.display import SVG
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer, base_filter
#from keras.utils.visualize_util import model_to_dot, plot
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)


In [3]:
df = pd.read_csv("../MagicDatasets/Magic_Pandas_DF")

/home/icarus/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
magic_cards = df.drop(df.columns[0], axis=1)

In [5]:
magic_cards

,artist,border,cmc,colorIdentity,colors,flavor,hand,id,imageName,layout,...,starter,subtypes,supertypes,text,timeshifted,toughness,type,types,variations,watermark
0,Pete Venters,NaN,7.0,[W],[White],"""The will of all, by my hand done.""",NaN,ab1ab474019e4e76c66e2b524d354cb7c3212616,ancestor's chosen,normal,...,NaN,"[Human, Cleric]",NaN,First strike (This creature deals combat damag...,NaN,4,Creature — Human Cleric,[Creature],NaN,NaN
1,Volkan Baga,NaN,5.0,[W],[White],Every tear shed is a drop of immortality.,NaN,0fb7efcdacb0ac4c73829aa08ddee55feeadb56f,angel of mercy,normal,...,NaN,[Angel],NaN,Flying\nWhen Angel of Mercy enters the battlef...,NaN,3,Creature — Angel,[Creature],NaN,NaN
2,Mark Zug,NaN,3.0,[W],[White],Only the warrior who can admit mortal weakness...,NaN,0b307d9f2d99b37aa659cf6708b7747d973b6107,angelic blessing,normal,...,NaN,NaN,NaN,Target creature gets +3/+3 and gains flying un...,NaN,NaN,Sorcery,[Sorcery],NaN,NaN
3,Jim Murray,NaN,5.0,[W],[White],The harmony of the glorious is a dirge to the ...,NaN,34b67f8cf8651964995bfec268498082710d4c6a,angelic chorus,normal,...,NaN,NaN,NaN,Whenever a creature enters the battlefield und...,NaN,NaN,Enchantment,[Enchantment],NaN,NaN
4,John Avon,NaN,2.0,[W],[White],"""The Ancestor protects us in ways we can't beg...",NaN,bf37b84059fe240e735e7bef03996bd32869947e,angelic wall,normal,...,NaN,[Wall],NaN,Defender (This creature can't attack.)\nFlying,NaN,4,Creature — Wall,[Creature],NaN,NaN
5,D. Alexander Gregory,NaN,3.0,[W],[White],Not all silences are easily broken.,NaN,28cf1fc88c72512df98530439d45d64f441546b5,aura of silence,normal,...,NaN,NaN,NaN,Artifact and enchantment spells your opponents...,NaN,NaN,Enchantment,[Enchantment],NaN,NaN
6,Justin Sweet,NaN,4.0,[W],[White],"""At the Reapportionment, Eagle begged to be hu...",NaN,c192224e04e5e4a4b4c9b7e294e634c9f67e6d8a,aven cloudchaser,normal,...,NaN,"[Bird, Soldier]",NaN,Flying (This creature can't be blocked except ...,NaN,2,Creature — Bird Soldier,[Creature],NaN,NaN
7,Matthew D. Wilson,NaN,4.0,[W],[White],The perfect antidote for a tightly packed form...,NaN,d3488c0a916bd922856473155a8b49ec23fc2217,ballista squad,normal,...,NaN,"[Human, Rebel]",NaN,"{X}{W}, {T}: Ballista Squad deals X damage to ...",NaN,2,Creature — Human Rebel,[Creature],NaN,NaN
8,Rebecca Guay,NaN,1.0,[W],[White],"Life is measured in inches. To a healer, every...",NaN,2ddd954180f6d0a14ea0724175f6d043b854a1d8,bandage,normal,...,NaN,NaN,NaN,Prevent the next 1 damage that would be dealt ...,NaN,NaN,Instant,[Instant],NaN,NaN
9,Rob Alexander,NaN,6.0,[W],[White],The cave floods with light. A thousand rays sh...,NaN,4a7bf234af44e07d9a159d4d89e96ce320919917,beacon of immortality,normal,...,NaN,NaN,NaN,Double target player's life total. Shuffle Bea...,NaN,NaN,Instant,[Instant],NaN,NaN


In [6]:
name = "Anafenza, the Foremost"
magic_cards[magic_cards['name']==name]

,artist,border,cmc,colorIdentity,colors,flavor,hand,id,imageName,layout,...,starter,subtypes,supertypes,text,timeshifted,toughness,type,types,variations,watermark
15877,James Ryman,NaN,3.0,"[W, B, G]","[White, Black, Green]","Rarely at rest on the Amber Throne, Anafenza a...",NaN,84575c908afe702205978e9aa7af3ba52283ca99,"anafenza, the foremost",normal,...,NaN,"[Human, Soldier]",[Legendary],"Whenever Anafenza, the Foremost attacks, put a...",NaN,4,Legendary Creature — Human Soldier,[Creature],NaN,Abzan
31386,James Ryman,NaN,3.0,NaN,"[White, Black, Green]","Rarely at rest on the Amber Throne, Anafenza a...",NaN,33fcbcfecd161da85cb777b3ad8c256b0c631c47,"anafenza, the foremost",normal,...,NaN,"[Human, Soldier]",[Legendary],"Whenever Anafenza, the Foremost attacks, put a...",NaN,4,Legendary Creature — Human Soldier,[Creature],NaN,NaN


In [7]:
magic_cards_fill = magic_cards.fillna(0.0)

In [8]:
magic_cards_fill.head()

,artist,border,cmc,colorIdentity,colors,flavor,hand,id,imageName,layout,...,starter,subtypes,supertypes,text,timeshifted,toughness,type,types,variations,watermark
0,Pete Venters,0,7.0,[W],[White],"""The will of all, by my hand done.""",0.0,ab1ab474019e4e76c66e2b524d354cb7c3212616,ancestor's chosen,normal,...,0,"[Human, Cleric]",0,First strike (This creature deals combat damag...,0,4,Creature — Human Cleric,[Creature],0,0
1,Volkan Baga,0,5.0,[W],[White],Every tear shed is a drop of immortality.,0.0,0fb7efcdacb0ac4c73829aa08ddee55feeadb56f,angel of mercy,normal,...,0,[Angel],0,Flying\nWhen Angel of Mercy enters the battlef...,0,3,Creature — Angel,[Creature],0,0
2,Mark Zug,0,3.0,[W],[White],Only the warrior who can admit mortal weakness...,0.0,0b307d9f2d99b37aa659cf6708b7747d973b6107,angelic blessing,normal,...,0,0,0,Target creature gets +3/+3 and gains flying un...,0,0,Sorcery,[Sorcery],0,0
3,Jim Murray,0,5.0,[W],[White],The harmony of the glorious is a dirge to the ...,0.0,34b67f8cf8651964995bfec268498082710d4c6a,angelic chorus,normal,...,0,0,0,Whenever a creature enters the battlefield und...,0,0,Enchantment,[Enchantment],0,0
4,John Avon,0,2.0,[W],[White],"""The Ancestor protects us in ways we can't beg...",0.0,bf37b84059fe240e735e7bef03996bd32869947e,angelic wall,normal,...,0,[Wall],0,Defender (This creature can't attack.)\nFlying,0,4,Creature — Wall,[Creature],0,0


In [9]:
# cols = [0,2,3,5,8,11,12,16,21,22,24,32,34,35,36]
cols = [32]
new_df = magic_cards_fill.iloc[:,cols]

In [10]:
str_df = new_df.astype(str)
str_df

,text
0,First strike (This creature deals combat damag...
1,Flying\nWhen Angel of Mercy enters the battlef...
2,Target creature gets +3/+3 and gains flying un...
3,Whenever a creature enters the battlefield und...
4,Defender (This creature can't attack.)\nFlying
5,Artifact and enchantment spells your opponents...
6,Flying (This creature can't be blocked except ...
7,"{X}{W}, {T}: Ballista Squad deals X damage to ..."
8,Prevent the next 1 damage that would be dealt ...
9,Double target player's life total. Shuffle Bea...


In [11]:
sentence_str = str_df.iloc[:,0].tolist()

In [12]:
chars = set(''.join(sentence_str))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 99


In [13]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = max([len(s) for s in sentence_str])
print ('maxlen: ', maxlen)

maxlen:  760


In [14]:
step = 3
sentences = []
next_chars = []
for s in sentence_str:
    for i in range(0, len(s) - 1, step):
        sentences.append(s[i])
        next_chars.append(s[i + 1])

# print('nb sequences:', len(sentences))

In [15]:
print('Vectorization...')
X = np.zeros((len(sentence_str), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentence_str), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentence_str):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [16]:
X.shape

(31705, 760, 99)

In [17]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [18]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

#     start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)
        generated = 'Fly'
        sentence = 'Fly'
        print('----- Generating with seed: "Fly"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1


In [ ]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 2):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

#     start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

#         generated = ''
#         sentence = text[start_index: start_index + maxlen]
#         generated += sentence
        generated = 'elder vampires'
        sentence = 'elder vampires'
        print('----- Generating with seed: "elder vampires"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()